# MODULO_02 IDENTIFICACION INTERLOCUTORES: AGENTE Y CLIENTE

In [1]:
# ---------------------------------------------
# ATENCION - FIJAR ESTAS VARIABLES ANTES DE EJECUTAR
# ---------------------------------------------

nombre_lote = "LOTE_20250613"

nombre_modulo = "MODULO_02"

# ---------------------------------------------
# Configuración del entorno (Colab y Local)
# ---------------------------------------------

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

print(f"Entorno detectado: {'Google Colab' if EN_COLAB else 'Local'}")
print(f"Ruta base: {ruta_base}")

lote_id = nombre_lote.replace("LOTE_", "")

Entorno detectado: Local
Ruta base: G:/Mi unidad/TFM_EVA_MARTIN/Modulos


In [2]:
# ------------------------------------
# Cargar utilidades comunes
# ------------------------------------
import sys
import os
ruta_config = os.path.join(ruta_base, "config.yaml")

if ruta_base not in sys.path:
    sys.path.append(ruta_base)
import yaml

# Cargar configuración desde el archivo YAML
with open(ruta_config, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

config = yaml.safe_load(open(ruta_config))

# Extraer bloque de parámetros (KeyError si falta alguna clave)
params = config["parametros"]

#HF_TOKEN = params["hf_token"]
MODELO_HF_COLAB = params["modelo_hf_colab"]
MODELO_HF_LOCAL = params["modelo_hf_local"]

#os.environ["HUGGINGFACE_TOKEN"] = HF_TOKEN

# Bloque de Prompts
prompts = config["prompts"]
PROMPT_COLAB = prompts["prompt_colab"]
PROMPT_LOCAL = prompts["prompt_local"]

import pandas as pd
import utilidades_comunes

In [3]:
# ------------------------------------
# PASO 1: Configurar logger
# ------------------------------------
# 1. Configurar logger
logger = utilidades_comunes.configurar_logger(nombre_modulo, ruta_logs=os.path.join(ruta_base, nombre_modulo, "logs"))

# 2. Inicializar entorno
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base, ruta_config, logger=logger)


2025-06-22 23:45:48,391 - INFO - 📁 Entorno inicializado para MODULO_02
2025-06-22 23:45:48,409 - INFO - 📂 Ruta entrada: G:/Mi unidad/TFM_EVA_MARTIN\Modulos\MODULO_01\./salida
2025-06-22 23:45:48,411 - INFO - 📂 Ruta salida: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_02\./salida
2025-06-22 23:45:48,416 - INFO - 📂 Ruta logs: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_02\./logs
2025-06-22 23:45:48,419 - INFO - 📂 Ruta ejemplos: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_02\./ejemplos
2025-06-22 23:45:48,423 - INFO - 🔗 Módulo anterior: MODULO_01
2025-06-22 23:45:48,426 - INFO - 🆔 Lote ID: 20250613


In [4]:
# ------------------------------------
# PASO 2: Cargar y analizar dataset de entrada
# ------------------------------------

patron_busqueda = os.path.join(
    entorno["ruta_entrada"],
    f"dataset_{entorno['nombre_modulo_anterior'].lower()}_{entorno['lote_id']}*.csv"
)

import glob
archivos_encontrados = glob.glob(patron_busqueda)

if not archivos_encontrados:
    raise FileNotFoundError(f"No se encontró archivo de entrada para el lote {nombre_lote} con patrón: {patron_busqueda}")

fichero_entrada = archivos_encontrados[0]
df_entrada = utilidades_comunes.cargar_dataset(fichero_entrada, logger=logger)

utilidades_comunes.mostrar_muestra_dataset(df_entrada, "dataset de entrada", logger=logger)
utilidades_comunes.guardar_muestra_dataset(df_entrada, "entrada", entorno["ruta_ejemplos"], logger=logger, n=5)


2025-06-22 23:45:51,804 - INFO - ✅ Dataset cargado desde G:/Mi unidad/TFM_EVA_MARTIN\Modulos\MODULO_01\./salida\dataset_modulo_01_20250613.csv (18 filas, 2 columnas)
2025-06-22 23:45:51,806 - INFO - --- Muestra de dataset de entrada (primeras 5 filas) ---
2025-06-22 23:45:51,808 - INFO - Filas totales: 18, Columnas totales: 2
2025-06-22 23:45:51,919 - INFO - 
| id_transcripcion                                                                 | transcripcion                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [5]:
# ---------------------------------------------
# Definición de los dos prompts y función de construcción
# ---------------------------------------------

def construir_prompt(texto):

    if EN_COLAB:
        return PROMPT_COLAB.format(texto=texto)
    else:
        return PROMPT_LOCAL.format(texto=texto)    
    
import logging
logger = logging.getLogger(__name__)

In [6]:
def extraer_etiquetado(generado):
    sep = "Ahora haz lo mismo con esta conversación:"
    if isinstance(generado, dict):
        texto = generado.get("generated_text", "")
    elif isinstance(generado, str):
        texto = generado
    else:
        texto = ""
    return texto.split(sep)[-1].strip() if sep in texto else texto.strip()

In [7]:
# ---------------------------------------------
# Paso 4: Procesamiento específico del módulo
# ---------------------------------------------
import os

# — 0) Deshabilitar TensorFlow por completo —
os.environ["USE_TF"] = "0"

import torch
import numpy as np
import traceback
from huggingface_hub import login

# — 1) Parche de compatibilidad para NumPy >= 1.24 —
#if not hasattr(np, 'object'):   np.object   = object
#if not hasattr(np, 'bool'):     np.bool     = bool
#if not hasattr(np, 'int'):      np.int      = int
#if not hasattr(np, 'typeDict'): np.typeDict = dict

import getpass
if EN_COLAB:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    if not HF_TOKEN:
        HF_TOKEN = getpass.getpass("Por favor, introduce tu token de Hugging Face para Colab: ")
else:
    HF_TOKEN = getpass.getpass("Por favor, introduce tu token de Hugging Face para ejecución local: ")

if HF_TOKEN:
    os.environ["HUGGINGFACE_TOKEN"] = HF_TOKEN
    login(token=HF_TOKEN)
    logger.info("🔓 Token cargado desde entrada del usuario")
else:
    logger.warning("⚠️ No se proporcionó un token de Hugging Face. Ejecuta `huggingface-cli login` o define la variable de entorno.")


# — 3) Liberar memoria GPU —
torch.cuda.empty_cache()

# — 4) Inicializar pipeline SOLO PyTorch (sin TF, sin accelerate) —
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
logger.info(f"🔄 Inicializando modelo LLaMA 3 en {'GPU' if device>=0 else 'CPU'}…")
try:
    generator = pipeline(
        "text-generation",
        model=MODELO_HF_LOCAL,
        device=device,
        framework="pt"            # fuerza PyTorch y evita import de TF/Keras
    )
    tokenizer = generator.tokenizer
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    logger.info("✅ Modelo LLaMA 3 cargado correctamente.")
except Exception as e:
    logger.critical("❌ Error crítico al cargar el modelo LLaMA 3", exc_info=True)
    raise RuntimeError("Fallo en carga de LLaMA 3") from e



In [ ]:
@utilidades_comunes.medir_tiempo
def procesamiento(df):
    resultados = []
    logger.info(f"🔍 Procesando {len(df)} transcripciones con etiquetado LLM…")
    for i, fila in df.iterrows():
        try:
            logger.debug(f"📄 Procesando fila {i} (len={len(fila['transcripcion'])} caracteres)…")
            prompt = construir_prompt(fila["transcripcion"])
            salida = generator(prompt, max_new_tokens=2500, do_sample=False)[0]
            etiquetado = extraer_etiquetado(salida)
            logger.debug(f"✅ Fila {i} procesada correctamente.")
            resultados.append(etiquetado)
        except Exception as e:
            logger.error(f"❌ Error fila {i}: {e}")
            logger.debug(traceback.format_exc())
            resultados.append("ERROR")
    df["texto_etiquetado"] = resultados
    logger.info("✅ Procesamiento finalizado.")
    return df

df_salida = procesamiento(df_entrada)

In [ ]:
# ------------------------------------
# PASO 4: Validación post-procesamiento
# ------------------------------------
# Este paso garantiza que el DataFrame resultante cumple las condiciones estructurales.
# Se puede extender con validaciones adicionales específicas del módulo.

utilidades_comunes.validar_integridad(df_salida, logger)


2025-06-09 09:18:01,674 - INFO - Validando integridad del DataFrame...
2025-06-09 09:18:01,678 - INFO - Filas: 6, Columnas: 2
2025-06-09 09:18:01,683 - INFO - Valores nulos por columna:\nid_transcripcion    0
transcripcion       0
dtype: int64


In [ ]:
# ------------------------------------
# PASO 5: Guardar dataset salida con nombre estándar
# ------------------------------------
nombre_salida = os.path.join(
    entorno["ruta_salida"],
    f"dataset_{nombre_modulo.lower()}_{entorno['lote_id']}.csv"
)

utilidades_comunes.guardar_dataset(df_salida, nombre_salida, logger=logger)


2025-06-09 09:18:05,679 - INFO - Dataset de salida guardado en: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_01\./salida\dataset_modulo_01_20250609_091805.csv


In [ ]:
# ------------------------------------
# PASO 6: Mostrar muestra final
# ------------------------------------
nombre_muestra = f"{nombre_modulo.lower()}_{entorno['lote_id']}"
utilidades_comunes.mostrar_muestra_dataset(df_salida, nombre_muestra, logger=logger)

logger.info(f"✅ Finalización del procesamiento del módulo {nombre_modulo}")
logger.info(f"📦 Dataset final disponible en: {nombre_salida}")


In [ ]:
# ------------------------------------
# PASO 7: Guardar muestra en carpeta /ejemplos
# ------------------------------------
nombre_muestra = f"{nombre_modulo.lower()}_{entorno['lote_id']}"
utilidades_comunes.guardar_muestra_dataset(
    df_salida,
    nombre_muestra,
    entorno["ruta_ejemplos"],
    logger=logger
)


2025-06-09 09:18:25,718 - INFO - Guardada muestra de salida en G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_01\./ejemplos/muestra_modulo_01.csv
